# Hatchet Query Language

This notebook explores [Hatchet](https://github.com/LLNL/hatchet) queries from its [**String-based Dialect**](https://hatchet.readthedocs.io/en/latest/query_lang.html), specifically **Category 1: Quantifier Capabilities**. The notebook covers different query types that are capable of matching various number of nodes from [GraphFrame](https://hatchet.readthedocs.io/en/latest/user_guide.html) objects.

In [7]:
# display documentation for Hatchet GraphFrame
from IPython.display import Markdown, display
display(Markdown("../common/documentation/hatchet-query-language.md"))

Hatchet supports eight different categories for the query language, as shown in Fig. 1.  

|Category ID|Category Description|
|:---------:|:-------------------|
|1          |Quantifier Capabilities|
|2          |String Equivalence and Regex Matching Predicates|
|3          |String Containment Predicates (contains, starts with, ends with)|
|4          |Basic Numeric Comparison Predicates (==, >, >=, etc.)|
|5          |Special Value Identification Predicates (NaN, Inf, None)|
|6          |Predicate Combination through Conjunction (AND)|
|7          |Predicate Combination through Disjunction and Complement (OR, NOT)|
|8          |Predicate Combination through Other Operations (e.g., XOR)|

**Figure 1**: A table of the Hatchet Query Language capabilities, distinguished into categories and their corresponding category ID.

Hatchet offers multiple interfaces to define queries with different trade-offs to verbosity and expressiveness. An entire catalog of queries, use cases, categories and capabilities can be found [here](https://docs.google.com/spreadsheets/d/1fKNlHmDJdDbnE4jyMcaFqdnw6ZSaexgm33rOcVAj0do/edit#gid=0).

Hatchet query language consumes a GraphFrame and a sequence of queries. Each query can comprise a **predicate** and a **quantifier**. Hatchet query language finds all **matching paths** from a provided GraphFrame. For example, in Fig.2, for the query (any with A or B), the output would comprise of 2 paths, [1, 2, 4] and [1, 3, 4].

![Graph frames and queries](../common/images/hatchet_query_graphframe.PNG)

**Figure 2**: A diagram to provide an overview of queries and an example of how queries filter GraphFrames.

***




In [8]:
# display documentation for object-based dialect
display(Markdown("../common/documentation/string-based-dialect-01.md"))

The **String-based Dialect** is a formal language that can be used to create queries using a syntax derived from [Cypher](https://dl.acm.org/doi/10.1145/3183713.3190657). Queries generated using the **String-based Dialect** contain two main syntactic pieces: a *MATCH* statement and a *WHERE* statement. The *MATCH* statement starts with the *MATCH* keyword and defines the quantifiers and variable names used to refer to query nodes in the predicates. The *WHERE* statement starts with the *WHERE* keyword and defines one or more predicates. 

## Category 1: Quantifier Capabilities

A valid hatchet query requires a **quantifier**. The accepted values for query node quantifiers in the **String-based Dialect** are:

1. `"."`: Match a single node
2. `"*"`: Match 0 or more nodes
3. `"+"`: Match 1 or more nodes
4. `Integer`: Match an exact number of nodes 


In [2]:
# display dataset information 
display(Markdown("../common/documentation/dataset-information.md"))

### Loading profile data as Hatchet GraphFrame

Hatchet queries are only defined on Hatchet GraphFrames. 
Obtaining a hatchet GraphFrame is straight forward:

1. Import hatchet
2. Use the appropriate reader for the profile/trace at hand

We first load a [Caliper](https://github.com/LLNL/Caliper) profile in JSON format, where Caliper is a performance profiling library developed by the Lawrence Livermore National Lab (LLNL).

This example profile is profiled from [LULESH (Livermore Unstructured Lagrangian Explicit Shock Hydrodynamics)](https://asc.llnl.gov/codes/proxy-apps/lulesh), a performance report data generated by Caliper. LULESH is a highly simplified application designed to solve the Sedov Blast problem, which is a standard hydrodynamics test problem. It performs a hydrodynamics stencil calculation using both MPI and OpenMP to achieve parallelism. 

This is an interesting profile because it covers a relatively large number of nodes (45 nodes) and spends considerable time in MPI communication routines.


In [ ]:
import hatchet as ht
gf = ht.GraphFrame.from_caliper("../../data/lulesh-16nodes/lulesh-annotation-profile-512cores.json")

In [3]:
# display GraphFrame information 
display(Markdown("../common/documentation/graph-tree-information.md"))

### Displaying a Hatchet GraphFrame
A compact overview of a hatchet GraphFrame can be obtained using the `gf.tree()` function. We use this throughout the notebook to display the differences between an original GraphFrame and the resulting GraphFrame after applying a query.

In [ ]:
print(gf.tree())

In [4]:
# display DataFrame information 
display(Markdown("../common/documentation/dataframe-information.md"))

### Displaying a DataFrame
An additional detail perspective can be obtained by viewing the underlying data using a **DataFrame**. A Hatchet **DataFrame** holds all the numerical and categorical data associated with each node. 

In [ ]:
gf.dataframe

In [5]:
# why use drop index levels
display(Markdown("../common/documentation/drop-index-information.md"))

### Dropping index levels

As a precursor to defining queries, we drop the index level of the GraphFrame using the `drop_index_levels()` Hatchet function. Hatchet hierarchical indexing can be of two types, depending on whether there is a single metric per node or multiple set of metrics per node.  

If a node contains a single metric, the DataFrame will use an `Index` object containing the node column. If a node has an additional level of information, Hatchet creates a `MultiIndex` to store the information pertaining to multiple sets of metrics per node. `MultiIndex` stores the node column as the "top" level of the index, followed by additional information on the levels below. 

Based on the types of indexing (`Index or MultiIndex`), retrieving data from a DataFrame corresponding to a particular node either retrieves a single or multiple rows. This difference can cause issues when applying query node predicates.
Therefore, it is necessary to get rid of all index levels besides the node column through an aggregation operation on the GraphFrame. Then, a query node predicate can be applied to the GraphFrame. 

In [15]:
gf.drop_index_levels()

In [3]:
# display query type 1 documentation
display(Markdown("../common/documentation/quantifier-capabilities-01.md"))

### Use Case 1: Match a single node

In the simplest case, a user can use this query type to match all single nodes that belong to function calls of a particular library (e.g., MPI). 

The following query matches all single nodes where the predicate that the `name` metric `starts with "MPI_Barrier"` is satisfied:


In [16]:
# single node with names starting with MPI_Barrier
query_1 = """
MATCH (".", p)
WHERE p."name"=~"MPI_Barrier"
"""

The above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame. The `filter()` function takes a user-supplied function or query object and applies that to all rows in the DataFrame. The resulting Series or DataFrame is used to filter the DataFrame to only return rows that are true.

In [17]:
gf_filt = gf.filter(query_1)

The resulting GraphFrame now only lists the  nodes that matched the query:

In [ ]:
print(gf_filt.tree())

The query match is also reflected in the DataFrame:

In [ ]:
gf_filt.dataframe

In [4]:
# display query type 2 documentation
display(Markdown("../common/documentation/quantifier-capabilities-02-01.md"))

### Use Case 2: Match zero or more nodes

In many cases, a user may not know how many nodes to match. For this reason, hatchet provides the `"*"` and `"+"` literals as a quantifier `match zero or more nodes` and `match one or more nodes`, respectively.

This query type filters a GraphFrame with the object syntax to find all query paths with zero or more nodes that meet a query predicate. This notebook contains two examples for this query use case. The purpose of the second example is to illustrate the difference between the query use cases that `match zero or more nodes` and `match one or more nodes`.

**Example 1:**

For the first example, the following query matches all zero or more nodes where the predicate that the `name` metric `starts with "Calc"` is satisfied.


In [22]:
# zero or more nodes with name starting with Calc
query_2_1 = """
MATCH ("*", p)
WHERE p."name"=~"Calc.*"
"""

Just as before, the above query is passed to Hatchet’s filter() function to filter the input GraphFrame.

In [23]:
gf_filt = gf.filter(query_2_1)

Here, instead of matching only single nodes, entire call stacks can be matched. The resulting GraphFrame now only lists the nodes that matched the query:

In [ ]:
print(gf_filt.tree())

The query match is also reflected in the DataFrame:

In [ ]:
gf_filt.dataframe

In [8]:
# display query type 2 documentation
display(Markdown("../common/documentation/quantifier-capabilities-02-02.md"))

**Example 2:**

In some cases, it is necessary to constrain which nodes to match. In other cases, it may be unknown which functions are called before a particular routine.

For this second example, the first quantifier (`"."`) constrains the filter to single node with the predicate that the metric `name`, `starts with lulesh`. The second quantifier (`"."`) all nodes matching any node, before only `matching zero or more nodes` that satisfy the predicate that the metric `name`, `starts with Calc`.  

In [28]:
# zero or more nodes with several query nodes
query_2_2 = """
MATCH (".", p)->(".")->("*", p1)
WHERE p."name"=~"lulesh.*" AND
    p1."name"=~"Calc.*"
"""

Just as before, the above query is passed to Hatchet’s filter() function to filter the input GraphFrame.

In [29]:
gf_filt = gf.filter(query_2_2)

The resulting GraphFrame now only lists the nodes that matched the query:

In [ ]:
print(gf_filt.tree())

In [9]:
# display query type 2 documentation
display(Markdown("../common/documentation/quantifier-capabilities-02-03.md"))

The above graph tree demonstrates that when the query to `match zero or more nodes` is executed with the constraints mentioned above, the node with `name "TimeIncrement"` is included, as it satisfies the third quantifier and predicate in the example. This specific node is omitted when the third quantifier in this example is changed to match one or more nodes instead. 

The query match is also reflected in the DataFrame: 


In [ ]:
gf_filt.dataframe

In [5]:
# display query type 3 documentation
display(Markdown("../common/documentation/quantifier-capabilities-03-01.md"))

### Use Case 3: Match one or more nodes

This query type filters a GraphFrame with the object syntax to find all query paths with one or more nodes that meet a query predicate. 

The notebook contains two examples for this query type. The purpose of the second example is to illustrate the difference between the query types that `match zero or more nodes` and `match one or more nodes`.

**Example 1:**

For the first example, the metric used for the query is `name` and the predicate is that the `name` metric `starts with "CalcMonotonic"`.

In [35]:
# one or more nodes with name starting with CalcMonotonic
query_3_1 = """
MATCH ("+", p)
WHERE p."name"=~"CalcMonotonic.*"
"""

Just as before, the above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame.

In [36]:
gf_filt = gf.filter(query_3_1)

The resulting GraphFrame now only lists the nodes that matched the query:

In [ ]:
print(gf_filt.tree())

The query match is also reflected in the DataFrame:

In [ ]:
gf_filt.dataframe

In [11]:
# display query type 3 documentation
display(Markdown("../common/documentation/quantifier-capabilities-03-02.md"))

**Example 2:**

For the second example, we repeat the second example in the previous section but replace the final query node to 'match one or more nodes'. The first quantifier (`"."`) constrains the filter to a single node with the predicate that the metric `name`, `starts with lulesh`. The second quantifier (`"."`) all nodes matching any node, before only `matching one or more nodes` that satisfy the predicate that the metric `name`, `starts with Calc`.  

In [40]:
# one or more nodes with several query nodes
query_3_2 = """
MATCH (".", p)->(".")->("+", p1)
WHERE p."name"=~"lulesh.*" AND
    p1."name"=~"Calc.*"
"""

Just as before, the above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame.

In [41]:
gf_filt = gf.filter(query_3_2)

The resulting GraphFrame now only lists the nodes that matched the query:

In [ ]:
print(gf_filt.tree())

In [12]:
# display query type 3 documentation
display(Markdown("../common/documentation/quantifier-capabilities-03-03.md"))

Execution of the second examples for query type 2 (match zero or more nodes) and query type 3 (match one or more nodes) demonstrate the difference between the two query types when used in combination with other quantifiers and predicates. 

The predicate that the metric `name`, `starts with Calc` for this dataset demonstrates that when we `match one or more nodes`, the filter omits the node with the `name "TimeIncrement"`, as it does not satisfy the third quantifier and predicate in this example. 

This specific node is included when the third quantifier in this example is changed to match zero or more nodes instead. The query match is also reflected in the DataFrame: 


In [ ]:
gf_filt.dataframe

In [6]:
# display query type 4 documentation
display(Markdown("../common/documentation/quantifier-capabilities-04.md"))

### Use Case 4: Match exact number of nodes


This query type filters a GraphFrame with the object syntax to find all query paths with an exact number of nodes, provided as an integer, that meets a query predicate.

The metric used for the query is `name` and the predicate is that the `name` metric `starts with "Calc"`. We have previously applied a query use case to match zero or more nodes that start with the name Calc. However, the user can use the following query to concisely match only those nodes that contain `exactly three nodes` that `start with name Calc`. The resulting GraphFrame should be relatively smaller, considering the original GraphFrame and the previous example.

In [46]:
# exactly three nodes with names starting with Calc
query_4 = """
MATCH (3, p)
WHERE p."name"=~"Calc.*"
"""

Just as before, the above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame.

In [47]:
gf_filt = gf.filter(query_4)

The resulting GraphFrame now only lists the nodes that matched the query:

In [ ]:
print(gf_filt.tree())

The query match is also reflected in the DataFrame:

In [ ]:
gf_filt.dataframe